In [8]:
sc

In [9]:
df = spark.read.load('joins_in_2015.csv', format='csv', header=True, inferSchema=False)
df.cache()

DataFrame[BOROUGH: string, CONTRIBUTING_FACTOR_VEHICLE_1: string, CONTRIBUTING_FACTOR_VEHICLE_2: string, CONTRIBUTING_FACTOR_VEHICLE_3: string, CONTRIBUTING_FACTOR_VEHICLE_4: string, CONTRIBUTING_FACTOR_VEHICLE_5: string, CROSS_STREET_NAME: string, DATETIME: string, LATITUDE: string, LOCATION: string, LONGITUDE: string, NUMBER_OF_CYCLIST_INJURED: string, NUMBER_OF_CYCLIST_KILLED: string, NUMBER_OF_MOTORIST_INJURED: string, NUMBER_OF_MOTORIST_KILLED: string, NUMBER_OF_PEDESTRIANS_INJURED: string, NUMBER_OF_PEDESTRIANS_KILLED: string, NUMBER_OF_PERSONS_INJURED: string, NUMBER_OF_PERSONS_KILLED: string, OFF_STREET_NAME: string, ON_STREET_NAME: string, UNIQUE_KEY: string, VEHICLE_TYPE_CODE_1: string, VEHICLE_TYPE_CODE_2: string, VEHICLE_TYPE_CODE_3: string, VEHICLE_TYPE_CODE_4: string, VEHICLE_TYPE_CODE_5: string, ZIP_CODE: string, _c28: string, Conditions: string, DateUTC: string, DatetimeEDT: string, Dew_PointF: string, Events: string, Freezing: string, Gust_SpeedMPH: string, Humidity: s

In [10]:
unique_locations = df.select("LOCATION").distinct()
unique_locations.count()

35408

In [11]:
locations = df.select("LOCATION")
locations.count()

217417

In [13]:
locations.groupBy("LOCATION").count().orderBy('count', ascending=False).count()

35408

In [14]:
df.select("LOCATION").groupBy("LOCATION").count().orderBy('count', ascending=False).show()

+--------------------+-----+
|            LOCATION|count|
+--------------------+-----+
|                null|34672|
|(40.8100528, -73....|  219|
|(40.6528929, -73....|  136|
|(40.6960346, -73....|  135|
|(40.7606005, -73....|  132|
|(40.7460781, -73....|  116|
|(40.7595273, -73....|  112|
|(40.6757357, -73....|  111|
|(40.7572323, -73....|  108|
|(40.7203211, -73....|  105|
|(40.7602226, -73....|  103|
|(40.6585778, -73....|   96|
|(40.7584292, -73....|   95|
|(40.8571285, -73....|   92|
|(40.8618642, -73....|   90|
|(40.6781627, -73....|   90|
|(40.7832478, -73....|   90|
|(40.8027515, -73....|   88|
|(40.759308, -73.9...|   88|
|(40.6631828, -73....|   87|
+--------------------+-----+
only showing top 20 rows



In [79]:
from pyspark.sql.functions import col

In [ ]:
overall_df = df.filter(col('LATITUDE').isNotNull() & ~col('LATITUDE').like('0'))\
        .select("LOCATION", 'LATITUDE', 'LONGITUDE').groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').count().orderBy('count', ascending=False)
overall_df.count()

In [48]:
rain_df = df.filter(col('Rain').like("1") & col('LATITUDE').isNotNull() & ~col('LATITUDE').like('0'))\
        .select("LOCATION", 'LATITUDE', 'LONGITUDE').groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').count().orderBy('count', ascending=False)
rain_df.count()

7053

In [49]:
snow_df = df.filter(col('Snow').like("1") & col('LATITUDE').isNotNull() & ~col('LATITUDE').like('0'))\
        .select("LOCATION", 'LATITUDE', 'LONGITUDE').groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').count().orderBy('count', ascending=False)
snow_df.count()

3347

In [50]:
windy_df = df.filter(col('Windy').like("1") & col('LATITUDE').isNotNull() & ~col('LATITUDE').like('0'))\
        .select("LOCATION", 'LATITUDE', 'LONGITUDE').groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').count().orderBy('count', ascending=False)
windy_df.count()

1105

In [51]:
inclement_df = df.filter((col('Rain').like("1") | col('Snow').like("1") | col('Windy').like("1") | \
           col('LowVisibility').like("1") | col('Freezing').like("1")) & \
           col('LATITUDE').isNotNull() & ~col('LATITUDE').like('0')) \
        .select("LOCATION", 'LATITUDE', 'LONGITUDE').groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').count().orderBy('count', ascending=False)
inclement_df.count()

16576

In [53]:
overall_df.coalesce(1).write.csv("2015_coords_for_carto/2015_overall_coords", header=True)
rain_df.coalesce(1).write.csv("2015_coords_for_carto/2015_rain_coords", header=True)
snow_df.coalesce(1).write.csv("2015_coords_for_carto/2015_snow_coords", header=True)
windy_df.coalesce(1).write.csv("2015_coords_for_carto/2015_windy_coords", header=True)
inclement_df.coalesce(1).write.csv("2015_coords_for_carto/2015_inclement_coords", header=True)
freezing_df.coalesce(1).write.csv("2015_coords_for_carto/2015_freezing_coords", header=True)

In [58]:
low_visibility_df = df.filter(col('LowVisibility').like("1") & col('LATITUDE').isNotNull() & ~col('LATITUDE').like('0'))\
        .select("LOCATION", 'LATITUDE', 'LONGITUDE').groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').count().orderBy('count', ascending=False)
low_visibility_df.count()

6024

In [57]:
low_visibility_df.coalesce(1).write.csv("2015_coords_for_carto/2015_low_visibility_coords", header=True)

In [94]:
rain_test = df.select("LOCATION", 'LATITUDE', 'LONGITUDE', df.Rain.cast("integer").alias("Rain")).groupBy("LOCATION", 'LATITUDE', 'LONGITUDE').sum('Rain').orderBy('sum(Rain)', ascending=False)
rain_test.show()

+--------------------+----------+-----------+---------+
|            LOCATION|  LATITUDE|  LONGITUDE|sum(Rain)|
+--------------------+----------+-----------+---------+
|                null|      null|       null|     1815|
|(40.8100528, -73....|40.8100528|-73.9251435|       11|
|(40.8285193, -73....|40.8285193|-73.8411794|       11|
|(40.6685062, -73....|40.6685062|-73.9256052|        9|
|(40.759308, -73.9...| 40.759308|-73.9652931|        9|
|(40.6694023, -73....|40.6694023|-73.9421971|        9|
|(40.7596199, -73....|40.7596199|-73.9619648|        8|
|(40.750766, -73.9...| 40.750766|-73.9744433|        7|
|(40.6482565, -73....|40.6482565|-73.9713978|        7|
|(40.6774132, -73....|40.6774132|-73.9303181|        7|
|(40.820189, -73.8...| 40.820189|-73.8906752|        7|
|(40.7139131, -73....|40.7139131|-73.8300825|        7|
|(40.7197624, -73....|40.7197624|-73.9447576|        7|
|(40.7584292, -73....|40.7584292|-73.9926328|        7|
|(40.8289603, -73....|40.8289603|-73.8420547|   

In [77]:
rain_test.count()

35408

In [89]:
df.registerTempTable('temp')

In [101]:
test = sqlContext.sql('select * from temp where LATITUDE = 40.7832478 and LONGITUDE = -73.9447234')
test.count()

90

In [102]:
test = sqlContext.sql('select * from temp where LATITUDE = 40.7832478 and LONGITUDE = -73.9447234 and rain = 1')
test.count()

7

In [135]:
join_test = overall_df.join(rain_test.select("LOCATION","sum(Rain)"), "LOCATION")
join_test.show()

+--------------------+----------+-----------+-----+---------+
|            LOCATION|  LATITUDE|  LONGITUDE|count|sum(Rain)|
+--------------------+----------+-----------+-----+---------+
|(40.8100528, -73....|40.8100528|-73.9251435|  219|       11|
|(40.6528929, -73....|40.6528929|-73.8667311|  136|        4|
|(40.6960346, -73....|40.6960346|-73.9845292|  135|        6|
|(40.7606005, -73....|40.7606005|-73.9643142|  132|        4|
|(40.7460781, -73....|40.7460781|-73.9749359|  116|        7|
|(40.7595273, -73....|40.7595273| -73.999242|  112|        6|
|(40.6757357, -73....|40.6757357|-73.8968533|  111|        3|
|(40.7572323, -73....|40.7572323|-73.9897922|  108|        1|
|(40.7203211, -73....|40.7203211|-73.9940403|  105|        6|
|(40.7602226, -73....|40.7602226| -73.967462|  103|        4|
|(40.6585778, -73....|40.6585778|-73.8906229|   96|        3|
|(40.7584292, -73....|40.7584292|-73.9926328|   95|        7|
|(40.8571285, -73....|40.8571285|-73.8807926|   92|        2|
|(40.783

In [154]:
rain_percents = join_test.select("*", ((join_test['sum(Rain)']/join_test['count'])*100).alias("rain%")).orderBy(['rain%','count'], ascending=False)
rain_percents.show()

+--------------------+----------+-----------+-----+---------+-----+
|            LOCATION|  LATITUDE|  LONGITUDE|count|sum(Rain)|rain%|
+--------------------+----------+-----------+-----+---------+-----+
|(40.7710869, -73....|40.7710869| -73.804419|    3|        3|100.0|
|(40.7076317, -73....|40.7076317| -73.915841|    3|        3|100.0|
|(40.7318009, -73....|40.7318009|-73.7170754|    2|        2|100.0|
|(40.7365763, -73....|40.7365763|-73.9077101|    2|        2|100.0|
|(40.6993586, -73....|40.6993586|-73.8423382|    2|        2|100.0|
|(40.5880534, -74....|40.5880534| -74.093535|    2|        2|100.0|
|(40.7658552, -73....|40.7658552|-73.9145003|    2|        2|100.0|
|(40.682097, -73.8...| 40.682097| -73.801709|    2|        2|100.0|
|(40.7200577, -74....|40.7200577|-74.0056566|    2|        2|100.0|
|(40.7660747, -73....|40.7660747|-73.7284567|    2|        2|100.0|
|(40.762987, -73.7...| 40.762987|-73.7285343|    2|        2|100.0|
|(40.8961082, -73....|40.8961082| -73.847742|   

In [141]:
rain_percents.registerTempTable('rain_temp')

In [151]:
test_percent = rain_percents.filter(col('rain%') > 50).filter(col('rain%') < 100)
test_percent.count()

42

In [155]:
test_percent.show(42)

+--------------------+----------+-----------+-----+---------+------------------+
|            LOCATION|  LATITUDE|  LONGITUDE|count|sum(Rain)|             rain%|
+--------------------+----------+-----------+-----+---------+------------------+
|(40.8229468, -73....|40.8229468|-73.8867266|    4|        3|              75.0|
|(40.6949675, -73....|40.6949675|-73.8083422|    4|        3|              75.0|
|(40.8409524, -73....|40.8409524| -73.879376|    4|        3|              75.0|
|(40.8289994, -73....|40.8289994|-73.8364102|    7|        5| 71.42857142857143|
|(40.7629027, -73....|40.7629027|-73.7691746|    3|        2| 66.66666666666666|
|(40.6808922, -73....|40.6808922|-73.8713658|    3|        2| 66.66666666666666|
|(40.7656519, -73....|40.7656519| -73.903572|    3|        2| 66.66666666666666|
|(40.6972977, -73....|40.6972977|-73.9237771|    3|        2| 66.66666666666666|
|(40.740402, -73.8...| 40.740402|-73.8821274|    3|        2| 66.66666666666666|
|(40.7046363, -73....|40.704